In [1]:
"""
这是数据采集文件
通过爬虫从百度体育网站爬取数据
"""
import re
# 导入必要模块
import urllib.error
import urllib.request

import pandas as pd
from bs4 import BeautifulSoup

# 正则提取格式
# 队名
findTN = re.compile(r'<div class="info c-color-link c-line-clamp1" data-a-9649c5d0="">(.*?)</div>')
# 数据
finddata = re.compile(r'<div class="score c-color-link" data-a-9649c5d0="">(.*?)</div>')
# 榜名
findRLN = re.compile(r'<div class="c-tabs-title" data-a-b498514c="">(.*?)</div>')

# 全局变量
# 排行榜名列表
rlnlist = []
# 数据列表
datalist = []
# 选手列表
playerlist = []


def askurl(url):
    # 请求头
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/100.0.4896.60 Safari/537.36 '
    }
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
    html = response.read().decode("utf-8")
    return html


def getdata(url):
    # 获取网页
    html = askurl(url)
    soup = BeautifulSoup(html, "html.parser")

    for item in soup.find_all('div', class_="match-team-group"):  # 把每一个排行榜从html文件中提取出来，形成列表
        item = str(item)  # 将每一项转化为字符串，以便正则提取

        # 正则提取，排行榜名
        rln = re.findall(findRLN, item)[0]

        # 正则提取，队名
        tn = re.findall(findTN, item)

        # 正则提取，数据
        data = re.findall(finddata, item)

        # 数据连接
        linkdata(rln, tn, data)


def linkdata(rln, tn, data):
    rlnlist.append(rln)
    datadict = dict(zip(tn, data))
    datalist.append(datadict)


def todataframe():
    alldata = dict(zip(rlnlist, datalist))
    df = pd.DataFrame(alldata)
    writer = pd.ExcelWriter('../data.xlsx')
    df.to_excel(writer, sheet_name='team data')
    writer.save()


if __name__ == '__main__':
    # 网页URL
    url = "https://tiyu.baidu.com/match/lpl/tab/%E6%88%98%E9%98%9F%E6%A6%9C/current/"
    # 获取数据
    getdata(url)
    todataframe()
